<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Evaluate the Gaussian integral:


In [ ]:
import numpy as np
from scipy.integrate import tanhsinh
def f(x):
    return np.exp(-x**2)
res = tanhsinh(f, -np.inf, np.inf)
res.integral  # true value is np.sqrt(np.pi), 1.7724538509055159

1.7724538509055159

In [ ]:
res.error  # actual error is 0

4.0007963937534104e-16

The value of the Gaussian function (bell curve) is nearly zero for
arguments sufficiently far from zero, so the value of the integral
over a finite interval is nearly the same.


In [ ]:
tanhsinh(f, -20, 20).integral

1.772453850905518

However, with unfavorable integration limits, the integration scheme
may not be able to find the important region.


In [ ]:
tanhsinh(f, -np.inf, 1000).integral

4.500490856616431

In such cases, or when there are singularities within the interval,
break the integral into parts with endpoints at the important points.


In [ ]:
tanhsinh(f, -np.inf, 0).integral + tanhsinh(f, 0, 1000).integral

1.772453850905404

For integration involving very large or very small magnitudes, use
log-integration. (For illustrative purposes, the following example shows a
case in which both regular and log-integration work, but for more extreme
limits of integration, log-integration would avoid the underflow
experienced when evaluating the integral normally.)


In [ ]:
res = tanhsinh(f, 20, 30, rtol=1e-10)
res.integral, res.error

(4.7819613911309014e-176, 4.670364401645202e-187)

In [ ]:
def log_f(x):
    return -x**2
res = tanhsinh(log_f, 20, 30, log=True, rtol=np.log(1e-10))
np.exp(res.integral), np.exp(res.error)

(4.7819613911306924e-176, 4.670364401645093e-187)

The limits of integration and elements of `args` may be broadcastable
arrays, and integration is performed elementwise.


In [ ]:
from scipy import stats
dist = stats.gausshyper(13.8, 3.12, 2.51, 5.18)
a, b = dist.support()
x = np.linspace(a, b, 100)
res = tanhsinh(dist.pdf, a, x)
ref = dist.cdf(x)
np.allclose(res.integral, ref)

True

By default, `preserve_shape` is False, and therefore the callable
`f` may be called with arrays of any broadcastable shapes.
For example:


In [ ]:
shapes = []
def f(x, c):
   shape = np.broadcast_shapes(x.shape, c.shape)
   shapes.append(shape)
   return np.sin(c*x)

c = [1, 10, 30, 100]
res = tanhsinh(f, 0, 1, args=(c,), minlevel=1)
shapes

[(4,), (4, 34), (4, 32), (3, 64), (2, 128), (1, 256)]

To understand where these shapes are coming from - and to better
understand how `tanhsinh` computes accurate results - note that
higher values of ``c`` correspond with higher frequency sinusoids.
The higher frequency sinusoids make the integrand more complicated,
so more function evaluations are required to achieve the target
accuracy:


In [ ]:
res.nfev

array([ 67, 131, 259, 515], dtype=int32)

The initial ``shape``, ``(4,)``, corresponds with evaluating the
integrand at a single abscissa and all four frequencies; this is used
for input validation and to determine the size and dtype of the arrays
that store results. The next shape corresponds with evaluating the
integrand at an initial grid of abscissae and all four frequencies.
Successive calls to the function double the total number of abscissae at
which the function has been evaluated. However, in later function
evaluations, the integrand is evaluated at fewer frequencies because
the corresponding integral has already converged to the required
tolerance. This saves function evaluations to improve performance, but
it requires the function to accept arguments of any shape.

"Vector-valued" integrands, such as those written for use with
`scipy.integrate.quad_vec`, are unlikely to satisfy this requirement.
For example, consider


In [ ]:
def f(x):
   return [x, np.sin(10*x), np.cos(30*x), x*np.sin(100*x)**2]

This integrand is not compatible with `tanhsinh` as written; for instance,
the shape of the output will not be the same as the shape of ``x``. Such a
function *could* be converted to a compatible form with the introduction of
additional parameters, but this would be inconvenient. In such cases,
a simpler solution would be to use `preserve_shape`.


In [ ]:
shapes = []
def f(x):
    shapes.append(x.shape)
    x0, x1, x2, x3 = x
    return [x0, np.sin(10*x1), np.cos(30*x2), x3*np.sin(100*x3)]

a = np.zeros(4)
res = tanhsinh(f, a, 1, preserve_shape=True)
shapes

[(4,), (4, 66), (4, 64), (4, 128), (4, 256)]

Here, the broadcasted shape of `a` and `b` is ``(4,)``. With
``preserve_shape=True``, the function may be called with argument
``x`` of shape ``(4,)`` or ``(4, n)``, and this is what we observe.